# RAG CON DEEPSEEK

Obiettivo di questo notebook è testare l'utilizzo del nuovo modello Deepseek-r1 in una pipeline di RAG.<br>
Per utilizzare localmente questo modello (in versione quantizzata 7B) utilizzo il tool Ollama.

TODO : da rivedere il modello utilizzato per embeddings (in italiano)
TODO : da approfondire l'utilizzo di Chroma (al momento se si esegue più volte aggiunge sempre documenti...)
TODO : L'estrazione dei documenti non sembra sempre congruente con la domanda in input

In [1]:
!ollama list

NAME                                   ID              SIZE      MODIFIED     
erwan2/DeepSeek-Janus-Pro-7B:latest    253d552064e2    4.2 GB    2 hours ago     
MFDoom/deepseek-r1-tool-calling:7b     2410129d448f    4.7 GB    2 days ago      
deepseek-r1:latest                     0a8c26691023    4.7 GB    3 days ago      
qwen2.5-coder:3b                       e7149271c296    1.9 GB    2 weeks ago     
sqlcoder:latest                        77ac14348387    4.1 GB    2 months ago    
llama3.2-vision:latest                 38107a0cd119    7.9 GB    2 months ago    
llama3.2:3b-instruct-fp16              195a8c01d91e    6.4 GB    4 months ago    
llama3.2:latest                        a80c4f17acd5    2.0 GB    4 months ago    


Per l'implementazione utilizzo:
- la libreria ollama
- langchain
- Chromadb : come vector database

In [2]:
#import gradio as gr
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
import ollama

Obiettivo del test è implementare una pipeline che, partendo da un pdf fornito in input, consenta l'interrogazione del pdf stesso al fine di rispondere a domande in linguaggio naturale riguardo il contenuto.

Il pdf è statico (nel codice) ma è possibile conventire le funzione in un'applicazione web per gestire l'upload del pdf di input.<br>
A tal fine si possono utilizzare librerie come Gradio o Streamlit

Innanzi tutto creo una funzione di gestire il pdf in input.<br>
La funzione riceve in input un pdf e predispone tutto il necessario per information retrieval.<br>


In [3]:
def process_pdf(pdf_bytes):
    if pdf_bytes is None:
        print("NO PDF")
        return None, None, None
    loader = PyMuPDFLoader(pdf_bytes)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_documents(data)
    print(f"Ottenuti : {len(chunks)} documenti")
    
    embeddings = OllamaEmbeddings(model="deepseek-r1")
    vectorstore=Chroma.from_documents(documents=chunks, embedding=embeddings)
    retriever = vectorstore.as_retriever()
    return text_splitter, vectorstore, retriever

Creo un'altra funzione che unisce i vari documenti estratti in un'unica stringa che verrà passata come contesto al modello per produrre la risposta finale

In [4]:
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

Questa funzione costituisce il cuore della pipeline.<br>
Riceve in input:
- la domanda posta dall'utente
- il contesto costituito dalla concatenazione dei documenti estratti dal vectore store sulla base delle similarity search con la domanda fornita

Utilizza queste 2 stringhe per creare il prompt da passare al modello deepseek-r1 in esecuzione su Ollama.
A seguito della risposta del modello, si procede ad eliminare i "thinking script" presenti nelle risposte di deepseek-r1 per ottenere la risposta finale

In [5]:
import re

def ollama_llm(question, context):

    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    print(f"Richiamo il modello con {formatted_prompt}")
    response = ollama.chat(model="deepseek-r1", messages=[{'role': 'user', 'content': formatted_prompt}])
    response_content = response['message']['content']
    
    # Remove content between <think> and </think> tags to remove thinking output
    final_answer = re.sub(r'<think>.*?</think>', '', response_content, flags=re.DOTALL).strip()
    return final_answer

Funzione di estrazione dei documenti dal vectore store sulla base di limilarity search con la domanda in input

In [6]:
def rag_chain(question, text_splitter, vectorstore, retriever):

    retrieved_docs = retriever.invoke(question)
    print(f"Estratti {len(retrieved_docs)} documenti")
    formatted_content = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_content)

Funzione principale che si occupa di gestire l'intera pipeline:
1) riceve il pdf di input e lo predispone per information retrieval
2) esegue la rag chain per ottenere la risposta finale

In [7]:
def ask_question(pdf_bytes, question):
    
    print("Processo il pdf in input")
    text_splitter, vectorstore, retriever = process_pdf(pdf_bytes)
    print("Pdf processato!")
    if text_splitter is None:
        print("nessun pdf caricato")
        return None  # No PDF uploaded
    print("Eseguo RAG Chain")
    result = rag_chain(question, text_splitter, vectorstore, retriever)
    return {result}

In [8]:
#pdf = "Preliminare.pdf"

In [9]:
#ask_question(pdf, "Come si chiama il notaio che ha redatto l'atto?")

In [10]:
pdf = "Maranello.pdf"
ask_question(pdf, "Qual'è il giorno di ingresso al museo?")

Processo il pdf in input
Ottenuti : 15 documenti
Pdf processato!
Eseguo RAG Chain
Estratti 4 documenti
Richiamo il modello con Question: Qual'è il giorno di ingresso al museo?

Context: Grazie per aver acquistato il biglietto di ingresso per la visita al nostro Museo.
Non è necessario stampare il presente biglietto, è sufficiente mostrarlo all’ingresso del Museo in formato 
elettronico.
Il biglietto ha validità nella data scelta in fase di acquisto e su di esso riportata.
L'ultimo accesso al Museo è consentito 45 minuti prima dell'orario di chiusura.
La visita ha la durata di un’ora circa.

Grazie per aver acquistato il biglietto di ingresso per la visita al nostro Museo.
Non è necessario stampare il presente biglietto, è sufficiente mostrarlo all’ingresso del Museo in formato 
elettronico.
Il biglietto ha validità nella data scelta in fase di acquisto e su di esso riportata.
L'ultimo accesso al Museo è consentito 45 minuti prima dell'orario di chiusura.
La visita ha la durata di un’or

{'The museum does not operate on a fixed "day of ingress." Instead, it offers entrance tickets that can be used on any date purchased during the visit. Each ticket is valid only on its specified purchase date and must be presented at the museum\'s electronic entry point upon arrival. Visitors are encouraged to check with the museum for their available dates and schedules as the ticket\'s validity period is tied directly to the chosen date of purchase.'}